In [1]:
import pandas as pd

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
features = pd.read_csv("features.csv")
stores = pd.read_csv("stores.csv")

In [3]:
train.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False


In [4]:
features.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [5]:
stores.head()

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


In [25]:
test.Date = pd.to_datetime(test.Date)
test["week"] = test.Date.dt.week

In [6]:
temp = train.merge(stores, on="Store")
temp = temp.merge(features, on=["Store", "Date"]).drop("IsHoliday_y", axis=1)

In [7]:
temp.drop("Size", axis=1, inplace=True)
temp.drop("Temperature", axis=1, inplace=True)
temp.drop("Type", axis=1, inplace=True)
temp.fillna(0, inplace=True)

In [12]:
temp["Date"] = pd.to_datetime(temp.Date)
temp["week"] = temp.Date.dt.week

In [13]:
y = temp['Weekly_Sales']
x = temp.drop('Weekly_Sales', axis=1)

In [15]:
temp.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday_x,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,week
0,1,1,2010-02-05,24924.50,False,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,5
1,1,2,2010-02-05,50605.27,False,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,5
2,1,3,2010-02-05,13740.12,False,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,5
3,1,4,2010-02-05,39954.04,False,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,5
4,1,5,2010-02-05,32229.38,False,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,5


In [29]:
gb = temp.groupby(["Store", "Dept", "week"]).mean()["Weekly_Sales"]

In [41]:
test = test.join(gb, on=["Store", "Dept", "week"], rsuffix = "_means")

In [42]:
test["key"] = test.Store.map(str) + '_' + test.Dept.map(str) + '_' + test.Date.map(str)

test.head()

,Store,Dept,Date,IsHoliday,week,Weekly_Sales,key,Weekly_Salesmeans_,Weekly_Sales_means
0,1,1,2012-11-02,False,44,37062.470,1_1_2012-11-02 00:00:00,37062.470,37062.470
1,1,1,2012-11-09,False,45,19119.465,1_1_2012-11-09 00:00:00,19119.465,19119.465
2,1,1,2012-11-16,False,46,19301.750,1_1_2012-11-16 00:00:00,19301.750,19301.750
3,1,1,2012-11-23,True,47,19865.770,1_1_2012-11-23 00:00:00,19865.770,19865.770
4,1,1,2012-11-30,False,48,23905.525,1_1_2012-11-30 00:00:00,23905.525,23905.525


In [45]:
cols = ["Weekly_Salesmeans_", "Weekly_Sales_means"]

test["Weekly_Sales2"] = test[cols].mean(axis=1)

In [46]:
test

,Store,Dept,Date,IsHoliday,week,Weekly_Sales,key,Weekly_Salesmeans_,Weekly_Sales_means,Weekly_Sales2
0,1,1,2012-11-02,False,44,37062.470000,1_1_2012-11-02 00:00:00,37062.470000,37062.470000,37062.470000
1,1,1,2012-11-09,False,45,19119.465000,1_1_2012-11-09 00:00:00,19119.465000,19119.465000,19119.465000
2,1,1,2012-11-16,False,46,19301.750000,1_1_2012-11-16 00:00:00,19301.750000,19301.750000,19301.750000
3,1,1,2012-11-23,True,47,19865.770000,1_1_2012-11-23 00:00:00,19865.770000,19865.770000,19865.770000
4,1,1,2012-11-30,False,48,23905.525000,1_1_2012-11-30 00:00:00,23905.525000,23905.525000,23905.525000
5,1,1,2012-12-07,False,49,32401.785000,1_1_2012-12-07 00:00:00,32401.785000,32401.785000,32401.785000
6,1,1,2012-12-14,False,50,45342.945000,1_1_2012-12-14 00:00:00,45342.945000,45342.945000,45342.945000
7,1,1,2012-12-21,False,51,51359.990000,1_1_2012-12-21 00:00:00,51359.990000,51359.990000,51359.990000
8,1,1,2012-12-28,True,52,21237.730000,1_1_2012-12-28 00:00:00,21237.730000,21237.730000,21237.730000
9,1,1,2013-01-04,False,1,16275.965000,1_1_2013-01-04 00:00:00,16275.965000,16275.965000,16275.965000
